# Smartphones and Watches advisor
In this serie of notebooks we will:
- Load text from a Youtube video playlist
- Split loaded text into appropriate chunks
- Create a vector store to load chunk embeddings
- **Create a Question Answering chatbot**:

    - with or without prompts engineering
    - with different similarity measures
    - **with** or without **chat memory**

!['Data Connection'](../../images/les_numeriques_youtube/data_connection.png)


# Create a Question Answering chatbot

Create a Question Answering memory chatbot 

In [16]:
# Imports 
# Env var
import os 
import sys
from dotenv import load_dotenv, find_dotenv

# Langchain
import openai
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# SelfQuery Retriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# Compressor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# QA retriever
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# Prompt template
from langchain.prompts import PromptTemplate

# Conversational chain 
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


In [17]:
# Env variable
sys.path.append('../..')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
# os.environ["LANGCHAIN_API_KEY"] = "[YOUR_API_KEY]"
# Go to https://smith.langchain.com/
# Setting > Create API key > Copy and past it in your.env file 

_ = load_dotenv(find_dotenv()) # read local .env file

In [18]:
# Load vector db
persist_directory = 'docs/chroma/'

embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

print(vectordb._collection.count())

815


In [19]:
# Load LLM
llm = OpenAI(temperature=0)

In [20]:
# Load compression retriever
compressor = LLMChainExtractor.from_llm(llm)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [21]:
# Build prompt
template = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 

Keep the answer as concise as possible. 
You can answer with bullet point as well.

Always say "Merci pour cette question!" at the end of the answer. 


CONTEXT: {context}
-------
Human: {question}
Assistant:

"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

In [22]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=compression_retriever,
    #retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, # Add a prompt
    verbose=True
)

In [23]:
question = "Quelles sont les caractéristiques de l'iPhone 14 ?"
result = qa_chain({"query": question})
print(result["result"])



> Entering new  chain...


/Users/florian.bastin/Documents/Code/Octo/GenAI/langchain_discovery/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.
L'iPhone 14 est équipé du processeur Apple A15 Bionic, a un écran de 6,7 pouces, une batterie améliorée, des capteurs présents sur les iPhone 13 Pro et un module frontal entièrement revu. Merci pour cette question!


In [24]:
question = 'Quel iPhone viens tu de me présenter cher LLM ?'

result = qa_chain({"query": question})
print(result["result"])



> Entering new  chain...

> Finished chain.
Je viens de te présenter l'iPhone 13 Product Red. Merci pour cette question!


### Conversational retrieval chain 


In [25]:
# Build prompt
template = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 

Keep the answer as concise as possible. 
You can answer with bullet point as well.

Always say "Merci pour cette question!" at the end of the answer. 


CONTEXT: {context}
-------
HISTORY: 
{chat_history}
Human: {question}
Assistant:

"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question", "chat_history"], template=template)

In [26]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [27]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

retriever = vectordb.as_retriever()
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
    verbose=True
)

In [28]:
question = "Quelles sont les caractéristiques de l'iPhone 14 ?"
result = qa_chain({"question": question})



> Entering new  chain...


> Entering new  chain...
Prompt after formatting:

Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 

Keep the answer as concise as possible. 
You can answer with bullet point as well.

Always say "Merci pour cette question!" at the end of the answer. 


CONTEXT: Comme prévu lors de sa canote de rentrée, Apple a annoncé non pas un, non pas deux, non pas trois, mais bien quatre iPhones. Pas d'iPhone mini au programme, mais ça c'était prévu. A la place on se retrouve avec un iPhone 14, un iPhone 14+, un iPhone 14 Pro et un iPhone 14 Pro Max et je vous propose qu'on décortique tout ça maintenant. On va commencer avec les deux entrées de gamme qui sont les iPhone 14 et les iPhone 14+.

Alors esthétiquement, il n'y a rien de nouveau par rapport aux iPhone 13 mini et aux iPhone 13 de l'année dernière. A l'intérieur, c'est quasiment la même chose, sau

In [29]:
print(result['answer'])

Les caractéristiques de l'iPhone 14 sont: 
- Écran de 6,7 pouces 
- Puce Apple A15 Bionic 
- Batterie améliorée 
- Capteurs photo du iPhone 13 Pro pour le principal et l'ultra grand-angle 
- Module frontal entièrement revu 
- Encoche et pas de Dynamic Island. 

Merci pour cette question!


In [30]:
question = "Quel iPhone viens tu de me présenter?"
result = qa_chain({"question": question})



> Entering new  chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Quelles sont les caractéristiques de l'iPhone 14 ?
Assistant: Les caractéristiques de l'iPhone 14 sont: 
- Écran de 6,7 pouces 
- Puce Apple A15 Bionic 
- Batterie améliorée 
- Capteurs photo du iPhone 13 Pro pour le principal et l'ultra grand-angle 
- Module frontal entièrement revu 
- Encoche et pas de Dynamic Island. 

Merci pour cette question!
Follow Up Input: Quel iPhone viens tu de me présenter?
Standalone question:

> Finished chain.


> Entering new  chain...


> Entering new  chain...
Prompt after formatting:

Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 

Keep the answer as concise as possible. 
You can answer with bullet point as well.

Alw

In [31]:
print(result['answer'])

L'iPhone dont je viens de vous présenter les caractéristiques est l'iPhone 14. 

Merci pour cette question!
